In [3]:
import os
import time as timer
from subprocess import Popen, PIPE

In [4]:
# First set the working directory to /home/pi/odas/bin
wd = "/home/pi/odas/bin"

In [5]:
# Using universal_newlines=True converts the output to a string instead of a byte array.
# matrix-odas only open once
p1 = Popen(["./matrix-odas", "&"], 
           cwd=wd, universal_newlines=True)

In [ ]:
def countdown():
    

In [10]:
countdown()

# Name the text file based on the recording time
path = "/home/pi/odas/recordings/"
filename = "SST.log"

while True:

    # Open the context manager
    with open(path+filename, "w") as output:    
        p2 = Popen(["./odaslive", "-vc", "../config/matrix-demo/matrix_creator_local.cfg"], 
                   stdout=output, 
                   cwd=wd,
                   universal_newlines=True)
    # The moment Odas starts to output log, start the timer    
    timer.sleep(290)
    p2.kill()
    p2.wait()
    
    # Obtain the last access time of raw files
    accessTime = os.path.getatime("/home/pi/odas/recordings/separated.raw")
    T = timer.strftime('%Y-%m-%d %H:%M:%S', timer.localtime(accessTime))
    date, time = str(T).split()
    time = time.split('.')[0]
    
    # Run odasparsing.py to check the SST log file
    p3 = Popen(['python', 'odasparsing.py'], stdout=PIPE, stdin=PIPE, universal_newlines=True)    
    output = p3.communicate(input=path+filename)[0]
    
    # Rename cleaned.log and remove SST.log
    os.rename("/home/pi/odas/recordings/cleaned.log", 
              "/home/pi/odas/recordings/cSST_" + date + "_" + time + ".log")
    # Delete SST.log
    os.remove("/home/pi/odas/recordings/SST.log")
    
    # If log file contains no data other than 0, delete raw files 
    if output is "not useful\n":
        os.remove("/home/pi/odas/recordings/separated.raw")
        os.remove("/home/pi/odas/recordings/postfiltered.raw")
    else:
        os.rename("/home/pi/odas/recordings/separated.raw", 
                  "/home/pi/odas/recordings/separated_" + date + "_" + time + ".raw")
        os.rename("/home/pi/odas/recordings/postfiltered.raw", 
                  "/home/pi/odas/recordings/postfiltered_" + date + "_" + time + ".raw")
        
    countdown()

In [9]:
#p2.returncode()
# Set by poll() and wait(). 
#p2.poll()
# If returns None, indicates that the process hasn’t terminated yet; if returncode, the process has terminated.

In [6]:
#!ls to check what files are in the current directory
#!pwd to check the path of the current directory